<a href="https://colab.research.google.com/github/allakoala/data_science/blob/main/colab_notebooks/HW_Regression_(basic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HW: https://docs.google.com/document/d/1er_E-XQrx9xsGEd0uNwCVfXqq_kU5YtHZLizHjJq2vA/edit?usp=sharing

#1. EDA: exploration of variables and properties of data

##Conclusions:
1. the following features were dropped because of the 97% on nan values: Unnamed: 16,Unnamed: 15, NMHC(GT)
2. the nan values in the following features should be inputed (mean and mode) as it less that. 1%: 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH'
3. All columns have no outliers.


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import math

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
plt.style.use('bmh')

from pylab import rcParams

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
url = "/content/drive/MyDrive/Data/AirQualityUCI.csv"

#read the file into a variable
data = pd.read_csv(url, sep=';')
data = data.drop_duplicates() #remove duplicates
data.head(25)

In [ ]:
#identify the data type
data.info()

In [ ]:
#for each dataset column print unique values
for col in data.columns:
    n_unique_values = data[col].nunique()
    unique_values = data[col].unique()
    print(f"{col}: {n_unique_values}: {unique_values}")

In [ ]:
#missing data for each variable and way to handle it. missing data can imply a reduction of the sample size

total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_2 = data.isna().sum().sort_values(ascending=False)
percent_2 = (data.isna().sum()/data.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent, total_2, percent_2], axis=1, keys=['Tota_null', 'Percent_null', 'Total_na', 'Percent_na'])
missing_data

In [ ]:
#Unnamed: 16,Unnamed: 15 delete
data=data.drop(['Unnamed: 16','Unnamed: 15'],axis=1)
data.head(5)

In [ ]:
# replace ',' with '.'
data = data.replace(',', '.', regex=True)

In [ ]:
# convert date and time columns to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
data['Time'] = pd.to_datetime(data['Time'], format='%H.%M.%S').dt.time

# merge date and time columns into a new column and set it as the index
data['DateTime'] = data['Date'].dt.strftime('%Y-%m-%d ') + data['Time'].astype(str)
data = data.set_index(pd.to_datetime(data['DateTime'], format='%Y-%m-%d %H:%M:%S'))

# extract desired features from date column
data['Day of week'] = data.index.dayofweek
data['Day number'] = data.index.day
data['Month number'] = data.index.month
data['Year'] = data.index.year
data['Season'] = (data.index.month % 12 + 3) // 3

# extract desired features from time column
data['Hour'] = data.index.hour
data['Part of day'] = np.where(data['Hour'].isin(range(6, 12)), 1,
                               np.where(data['Hour'].isin(range(12, 18)), 2,
                                        np.where(data['Hour'].isin(range(18, 24)) | data['Hour'].isin(range(0, 6)), 3, np.nan)))

# drop rows with missing values in Part of day column
data = data.dropna(subset=['Part of day'])

# drop unnecessary columns
data = data.drop(['Date', 'Time', 'DateTime'], axis=1)

data.head(5)

In [ ]:
# convert Part of day column to integer data type
data['Part of day'] = data['Part of day'].astype(int)
data['Hour'] = data['Hour'].astype(int)
data['Season'] = data['Season'].astype(int)
data['Year'] = data['Year'].astype(int)
data['Month number'] = data['Month number'].astype(int)
data['Day number'] = data['Day number'].astype(int)
data['Day of week'] = data['Day of week'].astype(int)

#convert O(GT), C6H6(GT) and RH, AH, T into float data type
data['CO(GT)'] = data['CO(GT)'].astype(float)
data['C6H6(GT)'] = data['C6H6(GT)'].astype(float)
data['RH'] = data['RH'].astype(float)
data['AH'] = data['AH'].astype(float)
data['T'] = data['T'].astype(float)

In [ ]:
#devide features into num and categirical

#empty lists for numerical and categorical features
numerical_cols = []
categorical_cols = []

#loop over each column and determine its data type
for col in data.columns:
    if data[col].dtype == 'object':
        categorical_cols.append(col)
    else:
        numerical_cols.append(col)

print("All columns:", data.columns)
print("Numerical features:", numerical_cols)
print("Categorical features:", categorical_cols)

In [ ]:
#categorical into dummy data
#cat_dummies_data = pd.get_dummies(data[categorical_cols]) #categorical input data (One-hot encode categorical features)
#cat_dummies_data.columns

In [ ]:
#outliers detection
# Interquartile Range (IQR) method (values outside the normal range)
for col in numerical_cols:
    vals = data[col].values
    q1, q3 = np.percentile(vals, [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outliers = vals[(vals < lower) | (vals > upper)]
    print(f"{col} has {len(outliers)} outliers")

In [ ]:
data = data.dropna()
data.info()

In [ ]:
# replace '-200' values with NaN so that we may inpute it
data = data.replace(['-200', -200, -200.000000], np.nan)
data.isna().sum()

In [ ]:
#missing data for each variable and way to handle it. missing data can imply a reduction of the sample size
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_2 = data.isna().sum().sort_values(ascending=False)
percent_2 = (data.isna().sum()/data.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent, total_2, percent_2], axis=1, keys=['Tota_null', 'Percent_null', 'Total_na', 'Percent_na'])
missing_data

In [ ]:
#handle missing values - NMHC(GT) - column drop, others with median and mode
# NMHC(GT) column drop
data = data.drop('NMHC(GT)', axis=1)

# median inputation
data = data.fillna(data.median())

# mode inputation
#data = data.apply(lambda x: x.fillna(x.mode()[0]))

print(data.head())

In [ ]:
#devide features into num and categirical

#empty lists for numerical and categorical features
numerical_cols = []
categorical_cols = []

#loop over each column and determine its data type
for col in data.columns:
    if data[col].dtype == 'object':
        categorical_cols.append(col)
    else:
        numerical_cols.append(col)

print("All columns:", data.columns)
print("Numerical features:", numerical_cols)
print("Categorical features:", categorical_cols)

##Numerical features analysis:
All list arу printed out after the plots in the code section below:
1. mean < median (distribution of values in that column is negatively skewed and it has a long tail on the left-hand side of the median and is more spread out towards the lower values. there are more observations with lower values than with higher values)
2. mean > median (distribution of values in that column is positevly skewed, presence of abnormal high values)
3. big difference between 75th %tile and max  and 25% and nim values (sign of extreme values-Outliers, check out box plots):
4. high standard deviation (values are more spread out and have a wider range of values)
5. low standard deviation (values are more tightly clustered around the mean and have a narrower range of values)
6. Feature pairs with positive slopes
7. Feature pairs with negative slopes
8. pairs of features which have a high positive correlation coefficient (close to 1), then they are likely measuring similar aspects of the data and may be redundant
9. pairs of features which have a high negative correlation coefficient (close to -1), then they may be measuring opposite aspects of the data and may be providing conflicting information

In [ ]:
#continuous variables distribution visualization using a histogram

#summary statistics of the numerical features
print(data[numerical_cols].describe())

#the histogram is colored by the target variable 'y' (which is binary) to see the differences in the distribution between the two target classes.
for col in numerical_cols:
    sns.histplot(data=data, x=col, kde=True)
    plt.show()

#lists of columns with specified characteristics
stats = data[numerical_cols].describe()
print(f"\nColumns with mean < median:\n{stats.columns[stats.loc['mean'] < stats.loc['50%']]}")
print(f"\nColumns with mean > median:\n{stats.columns[stats.loc['mean'] > stats.loc['50%']]}")
print(f"\nColumns with big difference between (75th %tile and max) or/and (25th %tile and min values):\n{stats.columns[((stats.loc['75%'] - stats.loc['max']).abs() > 100) | ((stats.loc['25%'] - stats.loc['min']).abs() > 100)]}")
print(f"\nColumns with high standard deviation:\n{stats.columns[stats.loc['std'] > 100]}")
print(f"\nColumns with low standard deviation:\n{stats.columns[stats.loc['std'] < 0.1]}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# create empty lists for positive and negative slopes
positive_slope_plots = []
negative_slope_plots = []

for i in range(len(numerical_cols)):
    for j in range(i+1, len(numerical_cols)):
        # calculate the slope of the regression line
        slope = data[numerical_cols[j]].corr(data[numerical_cols[i]])

        # create scatter plot with color-coded points
        sns.scatterplot(x=numerical_cols[i], y=numerical_cols[j], data=data)

        # add regression line
        sns.regplot(x=numerical_cols[i], y=numerical_cols[j], data=data, scatter=False, color="black")

        # set plot title and axis labels
        plt.title(f"{numerical_cols[i]} vs {numerical_cols[j]}")
        plt.xlabel(numerical_cols[i])
        plt.ylabel(numerical_cols[j])

        # append plot and feature pair to the appropriate list based on the sign of the slope
        if slope > 0:
            positive_slope_plots.append((f"{numerical_cols[i]} vs {numerical_cols[j]}", plt))
        elif slope < 0:
            negative_slope_plots.append((f"{numerical_cols[i]} vs {numerical_cols[j]}", plt))

        # display plot
        plt.show()

        # clear current figure to free up memory
        plt.clf()

# print out the lists of feature pairs
print("Feature pairs with positive slopes:")
for feature_pair, plot in positive_slope_plots:
    print(f"{feature_pair}: {plot}")

print("\nFeature pairs with negative slopes:")
for feature_pair, plot in negative_slope_plots:
    print(f"{feature_pair}: {plot}")

In [ ]:
#heatmap style
sns.set(style='darkgrid')
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, annot=True, fmt=".2f", cmap='coolwarm', vmax=.8, square=True)
plt.show()

# print highly positively correlated pairs
pos_corr_pairs = []
for i in range(len(corrmat.columns)):
    for j in range(i+1, len(corrmat.columns)):
        if abs(corrmat.iloc[i, j]) >= 0.5:
            pos_corr_pairs.append((corrmat.columns[i], corrmat.columns[j]))

print("Highly positively correlated pairs:")
for pair in pos_corr_pairs:
    print(pair)

# print highly negatively correlated pairs
neg_corr_pairs = []
for i in range(len(corrmat.columns)):
    for j in range(i+1, len(corrmat.columns)):
        if abs(corrmat.iloc[i, j]) <=  -0.5:
            neg_corr_pairs.append((corrmat.columns[i], corrmat.columns[j]))

print("Highly negatively correlated pairs:")
for pair in neg_corr_pairs:
    print(pair)

##Categorical features analusis:
1. there are no categorical features

In [ ]:
#frequency table of the categorical features

for col in categorical_cols:
  print(f"\n{col}:")
  print(data[col].value_counts())

#2. Data preparation: normalization of data / scaling

##Conclusions:
1.If the data has a Gaussian distribution and outliers, it might be more appropriate to use a method like the Z-score normalization or standardization, which scales the data to have zero mean and unit variance. This method subtracts the mean and divides by the standard deviation of the column to transform the data.

In [ ]:
#Standatization
from sklearn.preprocessing import StandardScaler

scaler_s = StandardScaler().fit(data[numerical_cols])
rescaled_d = scaler_s.transform(data[numerical_cols])
np.set_printoptions(precision=3)
print(rescaled_d)

In [ ]:
num_normalized_data_df = pd.DataFrame(rescaled_d, columns=numerical_cols)
num_normalized_data_df.head(15)

#3. Feature engineering - basic transformations (nonlinear)

##Conclusions:
1. The following features have been added: 'CO(GT)_sq', 'PT08.S1(CO)_sq', 'C6H6(GT)_sq','PT08.S2(NMHC)_sq', 'NOx(GT)_sq', 'PT08.S3(NOx)_sq', 'NO2(GT)_sq', 'PT08.S4(NO2)_sq', 'PT08.S5(O3)_sq', 'T_sq', 'RH_sq', 'AH_sq','Day of week_sq', 'Day number_sq', 'Month number_sq', 'Year_sq','Season_sq', 'Hour_sq', 'Part of day_sq', 'CO(GT)_cube','PT08.S1(CO)_cube', 'C6H6(GT)_cube', 'PT08.S2(NMHC)_cube','NOx(GT)_cube', 'PT08.S3(NOx)_cube', 'NO2(GT)_cube','PT08.S4(NO2)_cube', 'PT08.S5(O3)_cube', 'T_cube', 'RH_cube', 'AH_cube','Day of week_cube', 'Day number_cube', 'Month number_cube', 'Year_cube','Season_cube', 'Hour_cube', 'Part of day_cube', 'CO(GT)_sqrt','PT08.S1(CO)_sqrt', 'C6H6(GT)_sqrt', 'PT08.S2(NMHC)_sqrt','NOx(GT)_sqrt', 'PT08.S3(NOx)_sqrt', 'NO2(GT)_sqrt','PT08.S4(NO2)_sqrt', 'PT08.S5(O3)_sqrt', 'T_sqrt', 'RH_sqrt', 'AH_sqrt','Day of week_sqrt', 'Day number_sqrt', 'Month number_sqrt', 'Year_sqrt','Season_sqrt', 'Hour_sqrt', 'Part of day_sqrt', 'CO(GT)_log','PT08.S1(CO)_log', 'C6H6(GT)_log', 'PT08.S2(NMHC)_log', 'NOx(GT)_log','PT08.S3(NOx)_log', 'NO2(GT)_log', 'PT08.S4(NO2)_log','PT08.S5(O3)_log', 'T_log', 'RH_log', 'AH_log', 'Day of week_log','Day number_log', 'Month number_log', 'Year_log', 'Season_log','Hour_log', 'Part of day_log'

In [ ]:
# Apply square transformation to the data
num_data_squared = np.square(num_normalized_data_df)
num_data_squared.columns = [col + '_sq' for col in num_data_squared.columns]

# Apply cube transformation to the data
num_data_cubed = np.power(num_normalized_data_df, 3)
num_data_cubed.columns = [col + '_cube' for col in num_data_cubed.columns]

# Apply square root transformation to the data
num_data_sqrt = np.sqrt(num_normalized_data_df)
num_data_sqrt.columns = [col + '_sqrt' for col in num_data_sqrt.columns]

# Apply log transformation to the data
num_data_log = np.log(num_normalized_data_df)
num_data_log.columns = [col + '_log' for col in num_data_log.columns]

# Concatenate the original data with the transformed data
num_transformed_data = pd.concat([num_normalized_data_df, num_data_squared, num_data_cubed, num_data_sqrt, num_data_log], axis=1)
#num_transformed_data.head(15)
num_transformed_data.columns

#4. Baseline model - linear regression without regularization

##Conclusions:
1. The lower the value of the MSE, the better the performance of the model, and the closer the predicted values are to the actual values. This indicates that the linear regression model has performed well in predicting the target variable on the testing data.
2. All features with missing data is droped.

In [ ]:
# replacedata with the num_transformed_data data and cat_dummies_data
#data = pd.concat([cat_dummies_data, num_transformed_data], axis=1)
#data.head(15)

In [ ]:
#missing data for each variable
data = num_transformed_data
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_2 = data.isna().sum().sort_values(ascending=False)
percent_2 = (data.isna().sum()/data.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent, total_2, percent_2], axis=1, keys=['Tota_null', 'Percent_null', 'Total_na', 'Percent_na'])
missing_data

In [ ]:
data = data.dropna(axis=1)
data.info()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X = data.filter(regex='^(?!.*C6H6\(GT\)).*$')  # Select columns that do not contain "C6H6(GT)" in their name
y = data["C6H6(GT)"] # y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=42)

len(y) #check this

In [ ]:
# Create a linear regression object
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Use the model to make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the performance of the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)

In [ ]:
#from sklearn.datasets import make_regression

# Generate a regression problem https://www.datacamp.com/tutorial/tutorial-normal-equation-for-linear-regression
#X, y = make_regression(
#    n_samples=100,
#    n_features=2,
#    n_informative=2,
#    noise = 10,
#    random_state=25
#    )

# Visualize feature at index 1 vs target
#plt.subplots(figsize=(8, 5))
#plt.scatter(X[:, 1], y, marker='o')
#plt.xlabel("Feature at Index 1")
#plt.ylabel("Target")
#plt.show()

#5. Metrics chosen as well as reasoning behind each metric

##Conclusions:
1. The mean squared error (MSE) is a measure of the average squared difference between the predicted and actual values. A lower MSE indicates better performance of the model. In this case, the MSE is 1.939847787611042e-05, which is very low and indicates good performance.
2. The root mean squared error (RMSE) is the square root of the MSE and provides an estimate of the average deviation of the predictions from the actual values. The RMSE is 0.004404370315506, which is also very low, indicating that the model is making accurate predictions.
3. The mean absolute error (MAE) is another measure of the average difference between the predicted and actual values. Like the MSE and RMSE, a lower MAE indicates better performance. The MAE in this case is 0.0036665067855805476, which is very low and indicates good performance.
4. The R-squared (R2) score measures the proportion of variance in the dependent variable that can be explained by the independent variables. The R2 score ranges from 0 to 1, with higher values indicating better performance. In this case, the R2 score is 0.99998178102174, which is very high and indicates that the model is able to explain a large proportion of the variance in the dependent variable.

In [ ]:
# Evaluate the performance of the model using MSE, RMSE, MAE, and R2

rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_test - y_pred))
r2 = model.score(X_test, y_test)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)

#6. Feature importance, hyperparameters tuning

##Conclusions:
1. The output provides the RMSE value of the best model, which is 0.0044. This value represents the root mean squared error of the model's predictions on the test set. The lower the RMSE, the better the model's performance.
2. The feature importance is represented by the coefficients of the linear regression model. The coefficients with the largest absolute values are the most important features, positively or negatively impacting the target variable.
Looking at the coefficients, we can see that the "Year" feature has the largest negative coefficient, followed by "Year_cube" and "Part of day." This indicates that these features have the most significant negative impact on the target variable, while "PT08.S2(NMHC)," "PT08.S2(NMHC)_sq," and "PT08.S2(NMHC)_cube" have the largest positive coefficients and the most significant positive impact on the target variable.
3. Comparing the results of both models, we can see that Ridge regression has a slightly better performance than Lasso regression as it has a lower RMSE value. However, both models have produced highly accurate results, with R-squared scores indicating that the models explain the variability of the data almost perfectly.
the RMSE values are relatively small, which means that the model's predictions are very close to the actual values. This is a good indication that the model has learned the underlying patterns in the data and can make accurate predictions on new data.
  - The Lasso regression has found the best alpha value of 0.001, which results in an R-squared score of 0.999974 and an RMSE of 0.005.
  - The Ridge regression has found the best alpha value of 0.1, which results in an R-squared score of 0.999982 and an RMSE of 0.004.

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns

# Get the feature coefficients
coefficients = model.coef_

# Plot the feature coefficients
plt.figure(figsize=(10,6)) # Set the figure size to 10x6
sns.barplot(x=X_train.columns, y=coefficients)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Coefficients')
plt.title('Linear Regression Coefficients')
plt.show()

In [ ]:
# Tune hyperparameters using grid search
param_grid = {"fit_intercept": [True, False]}
grid_search = GridSearchCV(LinearRegression(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Refit the model using the best hyperparameters
best_model = LinearRegression(**grid_search.best_params_)
best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
print(f"RMSE (best model): {rmse_best}")

# Analyze residuals to check for model assumptions
residuals = y_test - y_pred_best
sns.histplot(residuals, kde=True)
plt.xlabel("Residuals")
plt.ylabel("Count")
plt.title("Residual Distribution")
plt.show()

# Sort feature coefficients in descending order of absolute magnitude
coefficients = best_model.coef_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})
feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values('Absolute Coefficient', ascending=False).reset_index(drop=True)
feature_importance.drop('Absolute Coefficient', axis=1, inplace=True)

# Display feature importance in descending order
print('Feature Importance (Descending Order):')
display(feature_importance)

# Plot the feature importances
plt.figure(figsize=(10,6))
sns.barplot(x=[f[:20] + '...' for f in feature_importance.Feature], y=feature_importance.Coefficient)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Coefficients')
plt.title('Linear Regression Coefficients')
plt.show()

In [ ]:
# Lasso model with regularization using alpha values from GridSearchCV
lasso = Lasso()
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
lasso_cv = GridSearchCV(lasso, param_grid, cv=5)
lasso_cv.fit(X_train, y_train)

# Fit Lasso model and make predictions on test set
lasso_preds = lasso_cv.predict(X_test)

# Evaluate Lasso model
print("\nLasso Regression Results:")
print(f"Best alpha value: {lasso_cv.best_params_['alpha']}")
print(f"R^2 score: {r2_score(y_test, lasso_preds)}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, lasso_preds))}")

# Ridge model with regularization using alpha values from GridSearchCV
ridge = Ridge()
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
ridge_cv = GridSearchCV(ridge, param_grid, cv=5)
ridge_cv.fit(X_train, y_train)

# Fit Ridge model and make predictions on test set
ridge_preds = ridge_cv.predict(X_test)

# Evaluate Ridge model
print("\nRidge Regression Results:")
print(f"Best alpha value: {ridge_cv.best_params_['alpha']}")
print(f"R^2 score: {r2_score(y_test, ridge_preds)}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, ridge_preds))}")